The images in the dataset were all different sizes. It was important to normalize them all to a desired dimension as well as omit anything that was below the desired dimension.

In [ ]:
import glob
import cv2

desired_dimension = 512
skipped_images = 0
kept_images = 0

for filepath in glob.iglob('images/*.jpg'):
    img = cv2.imread(filepath)
    if (img.shape[0] >= desired_dimension and img.shape[1] >= desired_dimension):
        kept_images += 1
        res_img = cv2.resize(img, dsize=(desired_dimension, desired_dimension), interpolation=cv2.INTER_AREA)
        cv2.imwrite(f'preprocessed_images_2/{str(kept_images)}.jpg', res_img)
    else:
        skipped_images +=1

print('Kept Images: ' + str(kept_images))
print('Skipped Images: ' + str(skipped_images))

In [6]:
import cv2
import numpy as np
from skimage.util import random_noise
from keras import Model, Input
from keras.layers import UpSampling2D
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers import Dense, Flatten
import tensorflow as tf

Functions for working with, processing, and showing the image data

In [7]:
def get_dataset(num_samples, noise_amount, orig_size, low_size, color_channels):
    # Initialize np array first, then reassign for better performance
    x = np.empty([num_samples, low_size, low_size, color_channels])
    y = np.empty([num_samples, orig_size, orig_size, color_channels])
    test_opencv = np.empty([num_samples, orig_size, orig_size, color_channels])
    for i in range(1, num_samples+1):
        img_orig = cv2.imread(f'preprocessed_images/{str(i)}.jpg')
        img_low_res = cv2.resize(img_orig, dsize=(low_size, low_size), interpolation=cv2.INTER_AREA)
        img_low_res_noise = random_noise(img_low_res, mode='s&p', amount=noise_amount)
        img_low_res_upscaled_test = cv2.resize(img_low_res_noise, dsize=(orig_size, orig_size), interpolation=cv2.INTER_AREA)

        # Adding the noise resizes the rgb values to 0-1
        x[i-1] = img_low_res_noise
        # But the orinal image is still 0-255
        y[i-1] = img_orig / 255.0
        test_opencv[i-1] = img_low_res_upscaled_test
    return x, y, test_opencv

def show_images(x_rgb, y_rgb, test_opencv_rgb, index):
    cv2.imshow('Image Before: ' + str(index), x_rgb[index])
    cv2.imshow('Image Desired: ' + str(index), y_rgb[index])
    cv2.imshow('Image Predicted: ' + str(index), out_rgb[index])
    cv2.imshow('OpenCV Prediction With Noise: ' + str(index), test_opencv_rgb[index])

def get_mse(x, y):
    a = x.flatten()
    b = y.flatten()
    return ((a - b)**2).mean()


The four models to initially test

In [8]:
def get_model_base(upscale_factor=2, channels=3):
    conv_args = {
        "activation": "relu",
        "padding": "same",
    }
    inputs = Input(shape=(None, None, channels))
    x = Conv2D(16, (3, 3), **conv_args)(inputs)
    x = Conv2D(32, (3, 3), **conv_args)(x)
    x = Conv2D(64, (3, 3), **conv_args)(x)
    x = Conv2D(64, (3, 3), **conv_args)(x)
    x = Conv2D(128, (3, 3), **conv_args)(x)
    x = Conv2D(channels * (upscale_factor ** 2), 3, **conv_args)(x)
    outputs = tf.nn.depth_to_space(x, upscale_factor)

    return Model(inputs, outputs)

def get_model_simplier(upscale_factor=2, channels=3):
    conv_args = {
        "activation": "relu",
        "padding": "same",
    }
    inputs = Input(shape=(None, None, channels))
    x = Conv2D(32, (3, 3), **conv_args)(x)
    x = Conv2D(64, (3, 3), **conv_args)(x)
    x = Conv2D(128, (3, 3), **conv_args)(x)
    x = Conv2D(channels * (upscale_factor ** 2), 3, **conv_args)(x)
    outputs = tf.nn.depth_to_space(x, upscale_factor)

    return Model(inputs, outputs)

def get_model_more_complex(upscale_factor=2, channels=3):
    conv_args = {
        "activation": "relu",
        "padding": "same",
    }
    inputs = Input(shape=(None, None, channels))
    x = Conv2D(16, (3, 3), **conv_args)(inputs)
    x = Conv2D(16, (3, 3), **conv_args)(inputs)
    x = Conv2D(32, (3, 3), **conv_args)(x)
    x = Conv2D(64, (3, 3), **conv_args)(x)
    x = Conv2D(64, (3, 3), **conv_args)(x)
    x = Conv2D(128, (3, 3), **conv_args)(x)
    x = Conv2D(256, (3, 3), **conv_args)(x)
    x = Conv2D(channels * (upscale_factor ** 2), 3, **conv_args)(x)
    outputs = tf.nn.depth_to_space(x, upscale_factor)

def get_model_max_pooling(upscale_factor=2, channels=3):
    conv_args = {
        "activation": "relu",
        "padding": "same",
    }
    inputs = Input(shape=(None, None, channels))
    x = Conv2D(16, (3, 3), **conv_args)(inputs)
    x = Conv2D(32, (3, 3), **conv_args)(x)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Conv2D(64, (3, 3), **conv_args)(x)
    x = Conv2D(64, (3, 3), **conv_args)(x)
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(128, (3, 3), **conv_args)(x)
    x = Conv2D(channels * (upscale_factor ** 2), 3, **conv_args)(x)
    outputs = tf.nn.depth_to_space(x, upscale_factor)

    return Model(inputs, outputs)

Running the models

In [11]:
def run_model(model, batch_size, epochs, x, y, test_opencv, open_images=False):
    # Finish model
    model.compile(optimizer='rmsprop',loss='mse')
    # Train the neural network
    model.fit(x=x, y=y, batch_size=batch_size, epochs=epochs)
    # Process model out back to np.uint8 type
    out = model.predict(x) * 255.0
    out_rgb = out.clip(0, 255).astype(np.uint8)
    y_rgb = (y * 255.0).astype(np.uint8)
    x_rgb = (x * 255.0).astype(np.uint8)
    test_opencv_rgb = (test_opencv * 255.0).astype(np.uint8)

    # Get metrics from model
    print('MSE of Model: ' + str(get_mse(y_rgb, out_rgb)))
    print('MSE of OpenCV Simple Resize: ' + str(get_mse(y_rgb, test_opencv_rgb)))

    if (open_images):
        # Show Example Images
        show_images(x_rgb, y_rgb, test_opencv_rgb, 0)
        show_images(x_rgb, y_rgb, test_opencv_rgb, 10)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
    
    return mse

# Image params
noise_amount = 0.0005
upscale_factor = 2
orig_size = 512
low_size = 256
color_channels = 3
# Model params
num_samples = 200
batch_size = 25
epochs = 100

x, y, test_opencv = get_dataset(num_samples, noise_amount, orig_size, low_size, color_channels)


===] - 1s 136ms/step - loss: 0.0086
Epoch 58/300
8/8 [==============================] - 1s 136ms/step - loss: 0.0072
Epoch 59/300
8/8 [==============================] - 1s 136ms/step - loss: 0.0075
Epoch 60/300
8/8 [==============================] - 1s 136ms/step - loss: 0.0094
Epoch 61/300
8/8 [==============================] - 1s 136ms/step - loss: 0.0117
Epoch 62/300
8/8 [==============================] - 1s 136ms/step - loss: 0.0091
Epoch 63/300
8/8 [==============================] - 1s 136ms/step - loss: 0.0081
Epoch 64/300
8/8 [==============================] - 1s 136ms/step - loss: 0.0052
Epoch 65/300
8/8 [==============================] - 1s 136ms/step - loss: 0.0116
Epoch 66/300
8/8 [==============================] - 1s 136ms/step - loss: 0.0086
Epoch 67/300
8/8 [==============================] - 1s 137ms/step - loss: 0.0092
Epoch 68/300
8/8 [==============================] - 1s 136ms/step - loss: 0.0072
Epoch 69/300
8/8 [==============================] - 1s 137ms/step - loss:

In [ ]:
model_base = get_model_base(upscale_factor, color_channels)
run_model(model_base, batch_size, epochs, x, y, test_opencv)

In [ ]:
model_complex = get_model_more_complex(upscale_factor, color_channels)
run_model(model_complex, batch_size, epochs, x, y, test_opencv)

In [ ]:
model_pooling = get_model_max_pooling(upscale_factor, color_channels)
run_model(model_pooling, batch_size, epochs, x, y, test_opencv)

In [ ]:
model_simple = get_model_simplier(upscale_factor, color_channels)
run_model(model_simple, batch_size, epochs, x, y, test_opencv)